# Modular arithmetic

## The greatest common divisor of two integers

An integer $k$ is said to divide another $n$ (written $k\mid n$) if there is some third integer $m$ such that $n = km$. A great property of the integers is that for any two integers $a, b$, there is some number $d$ called the greatest common divisor of $a$ and $b$ that divides both $d\mid a$ and $d\mid b$, as well as for any other $e$ with $e\mid a$ and $e\mid b$, we have $e\mid d$; this is what we mean by the "greatest" common divisor: any common divisor of $a$ and $b$ must also divide $d$.

Although the definition seems terse and convoluted, there is a very simple way of computing the greatest common divisor of two numbers! It is no complicated than the "long division" you've used all your life, it just extends the observations made about division a bit further.

### The Division algorithm and computing GCDs

The standard way of performing long division has a special name, the "Euclidean Division algorithm", and it is a way of obtaining the following result:

>**Theorem**: Given two integers $a, b$ with $b \neq 0$, there are unique (!) integers $q$ and $r$, with $0 \leq r < |b|$, such that
$$
a = qb + r
$$

Then, note for any $a$ and $b$, any common divisor $d$ of $a$ and $b$ also necessarily divides $r$, as $a - qb = r$ and $d\mid a, d\mid b$. Furthermore, since $d\mid b$ and $d\mid r$, we further observe $d\mid \gcd(b, r)$.
Then, we can keep repeating this process:
$$
\begin{align}
a &= q_1b + r_1\\
b &= q_2r_1 + r_2\\
r_1 &= q_3r_2 + r_3\\
\dots
\end{align}
$$

### ... ad infinitum?
Although there's a formal proof on why this terminates, you don't need a mathematical proof to justify yourself that 1 is a common divisor of every pair of numbers, and so this process stops at some point, i.e., we get that for some $r_n$
$$
r_{n-1} = q_{n-3}r_{n} + 0
$$
and $r_n$ is the greatest common divisor of $r_n$ and $q_{n-3}r_n$. Notice that since every common divisor of $a$ and $b$ was a common divisor of $r_k$ and $r_{k+1}$, every common divisor of $a$ and $b$ divides $r_n$, and $r_n$ is also a common divisor of $a$ and $b$. In other words, we have obtained a recursive algorithm for computing the greatest common divisor of two numbers!

In [1]:
def extendedGCD(x, y):
    if x < y:
        x, y = y, x
    div = lambda x, y: (x//y, x%y)
    factors = []

    # this is totally unnecessary, it just felt appropriate to relabel
    # the variables in a form similar to the DA: n = pq + r
    n, q = x, y

    while q > 0:
        fac, rem = div(n, q)
        factors.append(fac)
        n = q
        q = rem

    return n, factors
    

## The "ring"? of modular arithmetic
For some positive $n$, the integers modulo $n$ form what's called a _ring_, a mathematical abstraction specifically conceived to capture addition and multiplication together. We provide a friendly version of it here:

>**Definition**: A ring is a tuple $(R, +, \cdot, 0, 1)$, where $R$ is a set, $0\neq 1 \in R$ special elements, and $+:R\times R\to R$ and $\cdot: R\times R \to R$ functions, which satisfy the following:
>1. **Zero is the identity for addition**: For every $r\in R$, $r + 0 = r = 0 + r$
>1. **Addition is associative**: For every $r, s, t\in R$, $(r + s) + t = r + (s + t)$
>1. **Multiplication distributes over addition**: For every $r, s, t\in R$, $(r + s) \cdot t = rt + st$ and $t \cdot (r + s) = tr + ts$
>1. **Addition is commutative**: For every $r, s\in R$, $r + s = s + r$
>1. **_Multiplication is commutative_**\*: For every $r, s\in R$, $rs = sr$
>1. **_One is the identity for multiplication_**\*: For every $r\in R$, $r \cdot 1 = r = 1 \cdot r$

We use the notation $a + b = +(a, b)$ and $ab = a\cdot b = \cdot(a, b)$ for readability.

Some examples of rings include $\mathbb Z, \mathbb Q, \mathbb R, \mathbb C$ with the usual addition and multiplication and $\mathbb Z /n$ with the modulo addition and multiplication. If we were to relax some of the conditions (like commutativity or the existence of the element 1, we can get more examples, like the set of all square matrices with normal addition and matrix multiplication)

Technically speaking, with integer arithmetic and floating-point arithmetic already implemented in computers, you implicitly work with the rings $\mathbb Z$ and $\mathbb Q$ already without realizing it! We can go a step further and provide a basic implementation for $\mathbb Z /n$ as well:

In [2]:
class FiniteIntegerRing:
    def __init__(self, n):
        if n == 0:
            raise ValueError("Do not specify 0! Use normal operations instead!")
        elif n < 0:
            n = -n

        self._n = n
    
    def add(self, a, b):
        return (a + b) % self._n

    def sub(self, a, b):
        return self.add(a, -b)

    def id(self, a):
        return self.add(a, 0)

    def add_inv(self, a):
        return self.add(-a, 0)

    def mult(self, a, b):
        return (a * b) % self._n

    # Note that every ring has repeated multiplication, but in
    # the case of the integers, we can also think of it as exponentiation!
    def pow(self, b, e):
        c = 1
        for f in range(e):
            f += 1
            c = self.mult(b, c)
        return c
        

### Modular inverses
If you play around enough with various different rings $\mathbb Z /n$, you can see that $3\cdot 5 \equiv 1 \mod 7$, or $3\cdot 11 \equiv 1 \mod 16$, or $66\cdot 102 \equiv 1 \mod 127$ if you're really into playing around with numbers :o

So, just motivated by algebraic curiosity for now, for a given $x$, how can we find $a$ such that $ax \equiv 1 \mod n$ for n? In a more simplified language, how can we find inverses for elements in modular rings? Spend some time to answer this question.

**Hint**: Spend some time considering what's in common with the examples above, and also try solving the following:

1. What is the multiplicative inverse of $5$ in $\mathbb Z / 11$?
1. What is the multiplicative inverse of $5$ in $\mathbb Z / 10$?
1. More generally, when does an integer $k$ satisfy $k \equiv 0 \mod n$?

### Answers

1. $9\cdot 5 \equiv 44 + 1 \equiv 1 \mod 11$
1. If there were some inverse $a$ of $5$, then $5a \equiv 1 \mod 10$ would also result in
$$
2 \equiv 2\cdot 1 \equiv 2\cdot 5a \equiv 10a \equiv 0 \mod 10
$$
even though 10 does not divide 2, giving a contradiction.
1. Note that this question was just to entice a reaction after question 2; $k \equiv 0 \mod n$ exactly means $n \mid k$.


Now, it seems that not every number gets to have an inverse all the time. It seems that for $x$ to have an inverse in $\mathbb Z /n$, it needs to satisfy
$$
\begin{align*}
ax \equiv 1 \mod n \iff ax - 1 \equiv 0 \mod n\\
\text{Which also means}\ n \mid ax - 1 \iff ax - 1 = -bn \iff ax + bn = 1
\end{align*}
$$
Now then, notice that it is also sufficient that there be integers $a, b$ that satisfy this relation, as
$$
1 \equiv ax + bn \equiv ax \mod n
$$
Then, the problem reduces to finding two integers $a, b$ that satisfy the equation of that form. But what exactly does that equation tell us?

## The Bézout identity
For such integers $x$ and $n$, the Bézout identity is of the form $ax + bn = d$ for integers $a, b, d$. You can immediately make the observation that any common divisor of $x$ and $n$ necessarily divides $d$. The fascinating result that we can obtain is that there exists unique $s$ and $t$ such that the following also holds:
$$
sx + tn = \gcd(x, n)
$$
We have seen a bit of the proof already in [computing the GCD through the division algorithm](#the-division-algorithm-and-computing-gcds), namely the fact that we can obtain the gcd through repeated application of the division algorithm
$$
\begin{align}
a &= q_1b + r_1\\
b &= q_2r_1 + r_2\\
r_1 &= q_3r_2 + r_3\\
&\dots\\
r_{n-1} &= q_{n+1}r_n + 0
\end{align}
$$
Now, notice that each line can be rewritten as follows (we also rename $a := r_{-1}$ and $b := r_0$)
$$
\begin{align}
r_{-1} - q_1r_0 &= r_1\\
r_0 - q_2r_1 &= r_2\\
r_1 - q_3r_2 &= r_3\\
&\dots\\
r_{n-2} - q_nr_{n-1} &= r_n = \gcd(a, b)\\
r_{n-1} - q_{n+1}r_n &= 0
\end{align}
$$
The proof now boils down to a simple proof by induction: we claim that for every $k = -1, \dots, n-2$, we can form the expression $\lambda r_{k} + \gamma r_{k+1} = \gcd(a,b)$ for some integers $\lambda, \gamma$.

For $k = n-2$, this is obvious.

Suppose it holds for $k$; then, notice that we can make the rewrite
$$
\begin{align}
\lambda r_k &+ \gamma r_{k+1} = \gcd(a,b)\\
\lambda r_k &+ \gamma (r_{k-1} - q_{k+1}r_k) = \gcd(a,b)\\
\gamma r_{k-1} &+ (\lambda - q_{k+1})r_k = \gcd(a,b)
\end{align}
$$
and we obtain the desired result. One thing to note is that this simplifies the implementation significantly: if we keep track of all the $q_k$s involved in the GCD computation process, we can compute the integer coefficients of the Bézout identity just by applying the transformation
$$
(\lambda, \gamma) \mapsto (\gamma, \lambda - q_k)
$$
going backwards. (We can start with the expression $0 r_{n+1} + r_n = r_n$, i.e. $(\lambda, \gamma) = (0, 1)$.)

In [3]:
def pairBezout(x, y):
    coef = lambda P, c: (P[1], P[0] - P[1] * c)
    maxSorted = x < y
    d, factors = extendedGCD(x, y)

    # we throw out the last factor as we do not need it for divisibility
    factors = factors[::-1][1:]

    bezout = (0, 1)
    for fac in factors:
        bezout = coef(bezout, fac)

    if maxSorted:
        bezout = (bezout[1], bezout[0])

    return bezout

def testValidBezout(x, y):
    d, _ = extendedGCD(x, y)
    p, q = pairBezout(x, y)
    left = (p == 0) or (p * x % y == d)
    right = (q == 0) or (q * y % x == d)
    return all([left, right])


We can immediately infer two things:

1. $x$ has an inverse in $\mathbb Z/n$ if and only if $\gcd (x, n) = 1$, or weakly, for every $y$, there is an $a$ such that $ay \equiv \gcd(y, n) \mod n$,
1. Since for every prime number $p$, we have $\gcd(x, p) = 1$ for $0 \leq x < p$, every non-zero element of $\mathbb Z /p$ has an inverse.

**Note**: Structures with addition and multiplication defined so that every element has an additive inverse and every non-zero element has a multiplicative inverse, with both operations commutative and multiplication distributive over addition is called a _field_; fields are a natural setting for Linear Algebra!

All this note says is that $\mathbb Z /p$ is a structure that behaves like $\mathbb Q$, $\mathbb R$, $\mathbb C$!

In [4]:
def is_prime(n):
    from math import sqrt, ceil
    # we only need to check whether primes up to faclimit divide n, as some
    # factor needs to be smaller than faclimit (can be proven by
    # contradiction!) in fact, we can check whether any number in this range
    # divides n: this is way more expensive but for small numbers we don't have
    # to worry about finding primes inside the range 
    faclimit = ceil(sqrt(n))

    # we can divide the search in half just by testing for 2 and
    # limiting ourselves to odd primes
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for q in range(3, faclimit + 1, 2):
        if n % q == 0:
            return False

    return True

# find the closest prime sitting on top of low_seed
def generatePrime(low_seed):
    candidate = low_seed
    while True:
        if is_prime(candidate):
            return candidate
        else:
            candidate += 1

Given how every non-zero element in $\mathbb Z /p$ can have an inverse, we can specialize our rings to fields in the case of prime $p$. Play around with the following definition!

In [5]:
class FiniteField(FiniteIntegerRing):
    def __init__(self, p):
        if not is_prime(p):
            raise ValueError("Specified value {} should be prime!".format(p))
        self._n = p

    def mult_inv(self, a):
        if a == 0:
            raise ZeroDivisionError

        b, _ = pairBezout(self.id(a), self._n)
        return self.id(b)

    def div(self, a, b):
        return self.mult(a, self.mult_inv(b))


# RSA Cryptography

## Euler's Theorem

>**Theorem**: For coprime integers $a$ and $n$, $a^{\varphi(n)} \equiv 1 \mod n$.

*Proof*: $\varphi(n)$ denotes the number of integers $0 < x < n$ with $\gcd(x, n) = 1$, or equivalently, the number of invertible elements in the ring $\mathbb Z/n$. The set of all invertible elements in $\mathbb Z/n$ is denoted as ${\mathbb Z/n}^\times$. Notice that $1\in {\mathbb Z/n}^\times$ and for any two elements $x, y$ in this set, their product $xy$ is in the set as well.

There are only finitely many elements in this set, and so for some $k \in \mathbb N$, $a^k \equiv 1 \mod n$. There is a smallest such number, and so we can assume that it is $k$.

Now, for every $x\in {\mathbb Z/n}^\times$, consider the sets
$$
xA := \{ xa^m \mid m \in \mathbb N \}
$$
Now, for $x, y \in {\mathbb Z/n}^\times$, either $xA \cap yA = \emptyset$ or $xA\cap yA \neq \emptyset$. If the latter holds, for some $g$, we have $g \equiv xa^m \equiv ya^l \mod n$, and so $x \equiv ya^{l-m} \mod n$, meaning $xA \subset yA$, and also $y \equiv xa^{m-l} \mod n$, meaning $yA \subset xA$, and so the only two choices for elements $x, y$ is either $xA \cap yA = \emptyset$ or $xA = yA$. Then, the sets $xA$ partition ${\mathbb Z/n}^\times$.

Finally, for each $xA$, notice that it has exactly $k$ elements: $x1, xa, \dots, xa^{k-1}$, for if there were any fewer, it implies $xa^m \equiv xa^l$ for distinct $0 < m \neq l < k$ and $a^{m-l} \equiv 1 \mod n$, which contradicts that $k$ is the smallest such number.

Thus, ${\mathbb Z/n}^\times$ consists of sets $xA$, each of which has $k$ elements, i.e. $k \mid \varphi(n)$, i.e. $\varphi(n) = kM$ for some $M$ and so
$$
a^{\varphi(n)} \equiv a^{kM} \equiv (a^k)^M \equiv 1^M \equiv 1 \mod n
$$

## The Chinese Remainder Theorem
>**Theorem**: Let $m, n$ be coprime numbers. For every system
>$$
\begin{align*}
x &\equiv a_1 \mod m\\
x &\equiv a_2 \mod n
\end{align*}
$$
there is a unique solution in $0 \leq x < mn$.

For practical purposes, this means that instead of doing our computations in $\mathbb Z/mn$, which could be a pretty big ring, we can instead do two computations in $\mathbb Z/m$ and $\mathbb Z/n$ and combine the results using the method in the proof, yielding a much more manageable process.

*Proof of theorem*: Note that every $x\in \mathbb Z /mn$ is a solution of some system; just take the modulo in each system.

Note that a desired $x$ would have the following forms:
$$
\begin{align*}
x &= q_1m + a_1\\
x &= q_2n + a_2
\end{align*}
$$
and so, if we were able to recover either $q_1$ or $q_2$, we could reconstruct $x$.

Now, since $m$ and $n$ are coprime, they have the Bézout identity $\alpha m + \beta n = 1$ for $\alpha, \beta \in \mathbb Z$. With this information, define
$$
x := \beta n (a_1 - a_2) + a_2
$$
Notice then
$$
\begin{align*}
x &\equiv n \beta (a_1 - a_2) + a_2 \equiv a_2 \mod n\\
x &\equiv n \beta (a_1 - a_2) + a_2 \equiv (n\beta + m\alpha)(a_1 - a_2) + a_2 \equiv a_1 - a_2 + a_2 \equiv a_1 \mod m\\
\end{align*}
$$

Note that this immediately implies that for coprime $m$ and $n$, $\varphi(mn) = \varphi(m)\varphi(n)$, as the full implication shows that $\mathbb Z/mn \cong \mathbb Z/m \oplus \mathbb Z/n$ (it's completely fine if you don't know what this means, it'll suffice to use the result)

Try it out! Compute the following:
```
648236487326487236 % 36
```
by hand!

In [6]:
648236487326487236 % 36

8

## The math behind RSA
For two prime numbers $p$ and $q$ not equal to each other, we can form their product $n = pq$. Now, $n\mid m$ if and only if $p\mid m$ and $q\mid m$, since $p$ and $q$ are coprime. Then, note for any $0 \leq m < n$, we have that
$$
\begin{align*}
m(m^{\varphi(n)} - 1) \equiv 0 \mod p\\
m(m^{\varphi(n)} - 1) \equiv 0 \mod q\\
\end{align*}
$$
as either $p\mid m$, or $\gcd(p, m) = 1$ and so $m^{\varphi(p)} \equiv 1 \mod p$, and similarly for $q$. As a result,
$$
m^{\varphi(n)+1} \equiv m \mod n
$$

Now, if we were to find some $e$ which is coprime to $\varphi(n)$, and solve for its Bézout identity
$$
de + l\varphi(n) = 1
$$
we get $d$ which satisfies 
$$
\begin{align*}
0 \equiv m - m \equiv m^{ed + \gamma\varphi(n)} - m &\equiv m^{ed - 1}(m m^{\varphi(n)}) m^{(\gamma - 1)\varphi(n)} - m \equiv m^{ed - 1}(m)m^{(\gamma - 1)\varphi(n)} - m\\
&\equiv \dots \equiv m^{ed - 1}m - m \equiv m^{ed} - m \mod n
\end{align*}
$$
and since $p$ and $q$ are both prime, we have that $\varphi(p) = p-1, \varphi(q) = q-1$ and $\varphi(n) = \varphi(p)\varphi(q)$.

## The Chinese Remainder theorem

In [7]:
class RSAKey:
    def __init__(self, n, e):
        self.n = n
        self._key = e

        self._ring = FiniteIntegerRing(n)

    def encrypt(self, message):
        return self._ring.pow(message, self._key)

class RSASecretKey(RSAKey):
    def __init__(self, n, d, p, q):
        RSAKey.__init__(self, n, d)
        self.p = p
        self.q = q

        self._dp = d % (p - 1)
        self._dq = d % (q - 1)
        
        self._fp = FiniteField(p)
        self._fq = FiniteField(q)

    def decrypt(self, ciphertext):
        m1 = self._fp.pow(ciphertext, self._dp)
        m2 = self._fq.pow(ciphertext, self._dq)
        h = self._fp.mult(self._fp.mult_inv(self.q), m1 - m2)

        return m2 + h * q

In [8]:
def generateRSAKeyPair(p, q) -> (RSAKey, RSASecretKey):
    from math import lcm, gcd
    from numpy.random import choice
    n = p * q
    totient = lcm(p-1, q-1)

    # hardcoded limit; do it differently!
    coprimes = [x for x in range(2, min(totient, 1000))
                if gcd(x, totient) == 1]

    e = choice(coprimes)
    d, _ = pairBezout(e, totient)
    # getting rid of the negative
    d = d + totient

    pub = RSAKey(n, e)
    sec = RSASecretKey(n, d, p, q)
    return (pub, sec)

Let's encrypt a message and send it to ourselves :)

We generate primes in the range of $[768, 1024]$ so that we have somewhat large primes to work on without the computation taking ages.

In [9]:
from numpy.random import randint
p = generatePrime(randint(2**20, 2**21))
q = generatePrime(randint(2**20, 2**21))
print(p, q)

pub, sec = generateRSAKeyPair(p, q)

1448303 1564307


In [10]:
message = 0xbeef
ciphertext = pub.encrypt(message)
print(hex(ciphertext))

0x1a93fed18c3


In [11]:
decrypted_message = sec.decrypt(ciphertext)
print(hex(decrypted_message))

0xbeef


In [12]:
from math import log
# a given n can uniquely encode log_2(n) bits of information
int(log(p * q, 2))

41

### Proof-of-concept encryption scheme

In [13]:
class Stream:
    def pack(msg) -> int:
        num = 0
        data = bytes(msg, 'utf-8')
        for b in data:
            num = num << 8
            num += b

        return num

    def unpack(code) -> list:
        wrap = []
        while code != 0:
            b = code & 0xff
            wrap.append(b)
            code = code >> 8

        return wrap[::-1]


In [22]:
message = "Hello world!"

In [15]:
cipher_blocks = []

# the block size is how many bits can be losslessly encoded in a block;
# this is due to the Chinese remainder theorem
block_size = int(log(pub.n, 2))//8

for block in range(0, len(message), block_size):
    encoded = Stream.pack(message[block:block + block_size])
    cipher_block = pub.encrypt(encoded)
    cipher_blocks.append(cipher_block)

In [1]:
def decipherBlocks(cipher_blocks, sec):
    decoded_buffer = []
    for block in cipher_blocks:
        decipher = sec.decrypt(block)
        decoded_buffer += Stream.unpack(decipher)
    return decoded_buffer

In [2]:
%load_ext line_profiler
%lprun -f decipherBlocks decipherBlocks(cipher_blocks, sec)

ModuleNotFoundError: No module named 'line_profiler'

In [17]:
received_message = str(bytearray(decoded_buffer), 'utf-8')
print(received_message)
assert message == received_message


 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec ut quam nec ligula semper auctor. Donec ultricies odio erat, id ultricies nulla porta et. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Sed in tempor ex. Sed vel consectetur erat, ac convallis nibh. Pellentesque hendrerit purus sagittis, elementum lectus porttitor, mattis velit. Ut condimentum lobortis enim nec commodo. Donec ac libero auctor, dapibus ligula id, ultrices nulla. Nunc pretium eu velit nec laoreet. Curabitur ullamcorper lacus turpis, ac cursus urna accumsan nec. Curabitur luctus leo quam, id feugiat ipsum placerat at. Suspendisse tempus metus ac erat consectetur, ut ultricies orci maximus. Vestibulum tempor urna non odio gravida, id faucibus lectus finibus. Aenean a pellentesque diam. Duis in fermentum purus. 



# Elliptic-curve Cryptography

In [18]:
class Point:
    def __init__(self, x, y, is_infinity=False):
        if is_infinity:
            self.is_infinity = True
            self.x = 0
            self.y = 0
        else:
            self.is_infinity = False
            self.x = x
            self.y = y
            
    def tuple(self):
        return (self.x, self.y)

    def inf():
        return Point(0,0,is_infinity=True)


class Curve:
    def __init__(self, p, a, b):
        self.p = p
        self.a = a
        self.b = b
        self.field = FiniteField(p)
        self.curve = lambda pt: self.field.sub(self.field.pow(pt.y,2), 
                self.field.pow(pt.x,3) + self.field.mult(a,pt.x) + b
            )


    def add(self, P: Point, Q: Point) -> Point:
        f = self.field
        if P.is_infinity:
            return Q
        if Q.is_infinity:
            return P
            
        lmb = 0
        x1, y1 = P.tuple()
        x2, y2 = Q.tuple()
        
        if x1 == x2 and f.add(y1, y2) == 0:
            return Point.inf()
            
        if x1 != x2 or y1 != y2:
            lmb = (y2-y1) * f.mult_inv(x2 - x1)
        else:
            lmb = (3 * (x1 ** 2) + self.a) * f.mult_inv(2 * y1)
    
    
        x3 = f.sub((lmb ** 2), x1+x2)
        y3 = f.sub(lmb * (x1-x3), y1)
    
    
        return Point(x3, y3, False)

    def scalar_mult(self, n: int, P: Point) -> Point:
        if not n > 0:
            raise ValueError("The scalar should be greater than 0!")

        Q = P
        R = Point.inf()

        while n > 0:
            if n % 2 == 1:
                R = self.add(R, Q)
            Q = self.add(Q, Q)
            n = n//2
        return R


In [19]:
a = 497
b = 1768
p = 9739

In [20]:
P = Point(493, 5564)
Q = Point(1539, 4742)
R = Point(4403, 5202)

terms = [P, P, Q, R]
curve = Curve(p, a, b)

total = Point.inf()
for term in terms:
    total = curve.add(total, term)

total.tuple()


(4215, 2162)

In [21]:
X = Point(5323, 5438)
assert curve.scalar_mult(1337, X).tuple() == (1089, 6931)

In [28]:
P = Point(2339,2213)
ans = curve.scalar_mult(7863, P).tuple()
print(f"crypto{{%d,%d}}" % ans)

crypto{9467,2742}
